In [11]:
import os
import pandas as pd
import datetime
import numpy as np

path = "/home/mnowak/Downloads/piec/data_2021-04-19.csv"

data = pd.read_csv(path).drop(columns=['Unnamed: 0'])
data['Czas'] = pd.to_datetime(data['Czas'])
data["sec"] = data['Czas'].astype(np.int64) // 10**9
data = data.set_index('Czas')

data = data.drop(columns=["Nadawa_pylow", 'Strata_pasa', 'S600', 'S500', 'S00',
       'S100', 'Nadawa_koncentratu', 'Nadawa_1', 'Nadawa_2', 'Tlen_lanca',
       'Tlen_procesowe', 'Powietrze_procesowe', 'Przetlenienie',
       'Zawartosc_wegla', 'Zawartosc_siarki', 'Udzial_nadziarna',
       'Udzial_podziarna', 'Poziom_topu', 'Kat_dystrybucji',
       'Straty_odstojnika', 'Temperatura_zuzla', 'Drgania_kotla'])

data = data.rename(columns={"Powietrze_dystrybucyjne": "pow_dyst",
             "Zawartosc_tlenu": "zaw_tle",
             "Predkosc_dmuchu": "pre_dmu",
             "Strata_laczna": "str_lac"
             })

data

,pow_dyst,zaw_tle,pre_dmu,str_lac,sec
Czas,,,,,
2021-04-19 00:00:00,2700.0,81.0,50.0,23.298206,1618790400
2021-04-19 00:00:01,2700.0,81.0,50.0,23.303154,1618790401
2021-04-19 00:00:02,2700.0,81.0,50.0,23.308102,1618790402
2021-04-19 00:00:03,2700.0,81.0,50.0,23.313053,1618790403
2021-04-19 00:00:04,2700.0,81.0,50.0,23.318001,1618790404
...,...,...,...,...,...
2021-04-29 23:59:55,2700.0,81.0,45.0,23.419621,1619740795
2021-04-29 23:59:56,2700.0,81.0,45.0,23.417660,1619740796
2021-04-29 23:59:57,2700.0,81.0,45.0,23.415703,1619740797


In [12]:
# Normalizacja
POW_DYST_NORM = {"min": 1900.,"max": 3500.}
ZAW_TLE_NORM = {"min": 65., "max": 81.}
PRE_DMU = { "min": 40.,"max": 70.}
STR_LAC = { "min": 15.,"max": 30.}

df = data.copy()
df["pow_dyst"] = (df["pow_dyst"] - POW_DYST_NORM["min"]) / (POW_DYST_NORM["max"] - POW_DYST_NORM["min"])
df["zaw_tle"] = (df["zaw_tle"] - ZAW_TLE_NORM["min"]) / (ZAW_TLE_NORM["max"] - ZAW_TLE_NORM["min"])
df["pre_dmu"] = (df["pre_dmu"] - PRE_DMU["min"]) / (PRE_DMU["max"] - PRE_DMU["min"])
df["str_lac"] = (df["str_lac"] - STR_LAC["min"]) / (STR_LAC["max"] - STR_LAC["min"])
df["str_lac_sh"] = df["str_lac"].shift(-1000)


df

,pow_dyst,zaw_tle,pre_dmu,str_lac,sec,str_lac_sh
Czas,,,,,,
2021-04-19 00:00:00,0.5,1.0,0.333333,0.553214,1618790400,0.443324
2021-04-19 00:00:01,0.5,1.0,0.333333,0.553544,1618790401,0.443362
2021-04-19 00:00:02,0.5,1.0,0.333333,0.553873,1618790402,0.443399
2021-04-19 00:00:03,0.5,1.0,0.333333,0.554204,1618790403,0.443437
2021-04-19 00:00:04,0.5,1.0,0.333333,0.554533,1618790404,0.443453
...,...,...,...,...,...,...
2021-04-29 23:59:55,0.5,1.0,0.166667,0.561308,1619740795,NaN
2021-04-29 23:59:56,0.5,1.0,0.166667,0.561177,1619740796,NaN
2021-04-29 23:59:57,0.5,1.0,0.166667,0.561047,1619740797,NaN


In [13]:
# Filtrowanie
df = df.loc[:"2021-04-29 04:00:00"]

In [15]:
# Podział
pivot = int(len(df) * 0.75)
train = df.iloc[:pivot]
test = df.iloc[pivot:]
train

,pow_dyst,zaw_tle,pre_dmu,str_lac,sec,str_lac_sh
Czas,,,,,,
2021-04-19 00:00:00,0.5000,1.0,0.333333,0.553214,1618790400,0.443324
2021-04-19 00:00:01,0.5000,1.0,0.333333,0.553544,1618790401,0.443362
2021-04-19 00:00:02,0.5000,1.0,0.333333,0.553873,1618790402,0.443399
2021-04-19 00:00:03,0.5000,1.0,0.333333,0.554204,1618790403,0.443437
2021-04-19 00:00:04,0.5000,1.0,0.333333,0.554533,1618790404,0.443453
...,...,...,...,...,...,...
2021-04-26 14:59:55,0.0625,1.0,0.333333,0.660675,1619449195,0.647381
2021-04-26 14:59:56,0.0625,1.0,0.333333,0.660985,1619449196,0.647518
2021-04-26 14:59:57,0.0625,1.0,0.333333,0.661295,1619449197,0.647654


In [21]:
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

X = [train["pow_dyst"].values, train["zaw_tle"].values, train["pre_dmu"].values, train["str_lac"].values]
Y = train["str_lac_sh"]

look_back = 1000
model = Sequential()
model.add(LSTM(50, input_shape=(4, look_back)))
model.add(Dense(1))
model.compile(loss="mean_squared_error", optimizer="adam")
model.fit(X, Y, epochs=1000, batch_size=1000)

Epoch 1/1000


ValueError: in user code:

    /home/mnowak/PycharmProjects/CudoFlash/venv/lib/python3.6/site-packages/keras/engine/training.py:830 train_function  *
        return step_function(self, iterator)
    /home/mnowak/PycharmProjects/CudoFlash/venv/lib/python3.6/site-packages/keras/engine/training.py:813 run_step  *
        outputs = model.train_step(data)
    /home/mnowak/PycharmProjects/CudoFlash/venv/lib/python3.6/site-packages/keras/engine/training.py:770 train_step  *
        y_pred = self(x, training=True)
    /home/mnowak/PycharmProjects/CudoFlash/venv/lib/python3.6/site-packages/keras/engine/base_layer.py:989 __call__  *
        input_spec.assert_input_compatibility(self.input_spec, inputs, self.name)
    /home/mnowak/PycharmProjects/CudoFlash/venv/lib/python3.6/site-packages/keras/engine/input_spec.py:197 assert_input_compatibility  *
        raise ValueError('Layer ' + layer_name + ' expects ' +

    ValueError: Layer sequential_1 expects 1 input(s), but it received 4 input tensors

IndexError: tuple index out of range